In [1]:
from transformers import SegformerImageProcessor, AutoModelForSemanticSegmentation
from PIL import Image
import requests
import matplotlib.pyplot as plt
import torch.nn as nn
from torchinfo import summary
import torch
import numpy as np
import torch.optim as optim
import torchvision.transforms as T
import torch.nn.functional as F
from collections import defaultdict
import cv2
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
device = torch.device("cuda:5" if torch.cuda.is_available() else "cpu")
tf=T.ToTensor()


2024-06-16 02:40:24.847735: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
params={'image_size':512,
        'lr':2e-4,
        'beta1':0.5,
        'beta2':0.999,
        'batch_size':16,
        'epochs':300,}

In [3]:
image1=np.load('../../data/cv0_ori.npy')
# image2=np.load('../../data/cv1_ori.npy')
# image3=np.load('../../data/cv2_ori.npy')
# image4=np.load('../../data/cv3_ori.npy')
# image5=np.load('../../data/cv4_ori.npy')
mask1=np.load('../../data/cv0_mask.npy')
# mask2=np.load('../../data/cv1_mask.npy')
# mask3=np.load('../../data/cv2_mask.npy')
# mask4=np.load('../../data/cv3_mask.npy')
# mask5=np.load('../../data/cv4_mask.npy')

In [7]:
np_data={'image1':image1,'image2':image1,'image3':image1,'image4':image1,'image5':image1,'mask1':mask1,'mask2':mask1,'mask3':mask1,'mask4':mask1,'mask5':mask1}

In [36]:
class CustomDataset(Dataset):
    def __init__(self, image_list, label_list):
        self.img_path = image_list
        self.label = label_list

    def __len__(self):
        return len(self.img_path)

    def __getitem__(self, idx):
        image_path = self.img_path[idx]
        image_path=tf(cv2.cvtColor(image_path, cv2.COLOR_GRAY2RGB))
        
        label_path = self.label[idx]
        label_path = tf(cv2.resize(label_path, (128, 128)))
       
        return image_path, label_path

train_dataset = CustomDataset(image1, mask1)

val_dataset = CustomDataset(image1, mask1)
train_dataloader = DataLoader(
    train_dataset, batch_size=params['batch_size'], shuffle=True, drop_last=True)
validation_dataloader = DataLoader(
    val_dataset, batch_size=params['batch_size'], shuffle=True, drop_last=True)

In [37]:
def dice_loss(pred, target, num_classes=4):
    smooth = 1.
    dice_per_class = torch.zeros(num_classes).to(pred.device)

    for class_id in range(num_classes):
        pred_class = pred[:, class_id, ...]
        target_class = target[:, class_id, ...]

        intersection = torch.sum(pred_class * target_class)
        A_sum = torch.sum(pred_class * pred_class)
        B_sum = torch.sum(target_class * target_class)

        dice_per_class[class_id] = 1 - \
            (2. * intersection + smooth) / (A_sum + B_sum + smooth)

    return torch.mean(dice_per_class)


model = AutoModelForSemanticSegmentation.from_pretrained("mattmdjaga/segformer_b2_clothes",num_labels=4,ignore_mismatched_sizes=True).to(device)
optimizer = optim.Adam(
    filter(lambda p: p.requires_grad, model.parameters()), lr=params['lr'], betas=(params['beta1'], params['beta2']))


Some weights of SegformerForSemanticSegmentation were not initialized from the model checkpoint at mattmdjaga/segformer_b2_clothes and are newly initialized because the shapes did not match:
- decode_head.classifier.bias: found shape torch.Size([18]) in the checkpoint and torch.Size([4]) in the model instantiated
- decode_head.classifier.weight: found shape torch.Size([18, 768, 1, 1]) in the checkpoint and torch.Size([4, 768, 1, 1]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [41]:

metrics = defaultdict(float)
for k in range(5):
    val_loss=1000
    for epoch in range(300):
        train = tqdm(train_dataloader)
        count = 0
        running_loss = 0.0
        acc_loss = 0
        for x, y in train:
            model.train()
            y = y.to(device).float()
            count += 1
            x = x.to(device).float()
            optimizer.zero_grad()  # optimizer zero 로 초기화
            predict = model(x).logits.to(device)
            cost = dice_loss(predict, y)  # cost 구함
            acc = 1-dice_loss(predict, y)
            cost.backward()  # cost에 대한 backward 구함
            optimizer.step()
            running_loss += cost.item()
            acc_loss += acc
            train.set_description(
                f"epoch: {epoch+1}/{300} Step: {count+1} dice_loss : {running_loss/count:.4f} dice_score: {1-running_loss/count:.4f}")
            
        with torch.no_grad():
            val = tqdm(validation_dataloader)
            count = 0
            running_loss = 0.0
            acc_loss = 0
            for x, y in val:
                model.eval()
                y = y.to(device).float()
                count += 1
                x = x.to(device).float()
                predict = model(x).logits.to(device)
                cost = dice_loss(predict, y)
                acc = 1-dice_loss(predict, y)
                running_loss += cost.item()
                acc_loss += acc
                val.set_description(
                    f"epoch: {epoch+1}/{300} Step: {count+1} dice_loss : {running_loss/count:.4f} dice_score: {1-running_loss/count:.4f}")
                if val_loss>running_loss/count:
                    val_loss=running_loss/count
                    torch.save(model.state_dict(), '../../model/segformer/seg_former_'+str(k+1)+'_check.pth')
    torch.save(model.state_dict(), '../../model/segformer/seg_former_'+str(k+1)+'.pth')
    

epoch: 1/300 Step: 85 dice_loss : 0.3961 dice_score: 0.6039:  14%|█▍        | 84/610 [01:09<07:15,  1.21it/s]


KeyboardInterrupt: 

In [40]:
model(x).logits.shape

torch.Size([16, 4, 128, 128])